<a href="https://colab.research.google.com/github/Teasotea/textSummarization/blob/main/TextSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [204]:
!pip install transformers sentencepiece

# Load Data

In [205]:
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [206]:
conversation_df = pd.read_csv('https://raw.githubusercontent.com/Teasotea/textSummarization/main/data/session.csv', delimiter = "\t", header=None)
conversation_df = conversation_df.set_axis(['text'], axis=1, inplace=False)
conversation_df.style.set_properties(**{'text-align': 'left'})

,text
0,"Психолог: Добрый день, меня зовут Ирина. Ольга, у нас с вами на сегодня назначена консультация. Надеюсь, что наш разговор будет приятным и полезным. Расскажите с каким запросом вы пришли на нашу первую встречу?"
1,"Ольга: Добрый день, это моя первая консультация в жизни. Я не особо понимаю что именно нужно говорить. Но у меня накопилось достаточно много всего и мне просто хотелось поговорить о том, что сейчас происходит с моей жизнью, ну много чего, конечно, как и у всех, все таки война, переезды, куча всего короче. Просто хотелось чтобы кто-то услышал, помог мне немного разобраться в себе."
2,"Психолог: Ольга, действительно сейчас многие проживают события к которым сложно быть готовым. Давайте начнем понемногу разбираться. Расскажите о себе. Где вы сейчас находитесь, в каких условиях живете, какие события происходят вокруг вас?"
3,"Ольга: Сейчас я чувствую себя относительно в безопасности. Мы переехали из Киева на западную Украину. Здесь можно сказать, что тихо, но я не особо понимаю что происходит. Мне словно приходится начинать все с нуля. Деньги немного есть, ээээ ну там на пару месяцев хватит, но нужно искать работу. Не пойму нужно учится на новую специальность или попытаться найти работу менеджером, как раньше? Хотя кому эти менеджеры нужны, ничего толком не работает, наверно повезло людям которые сохранили работу, прям как-то радостно за них. Потерять финансовую стабильность мне реально тяжело, ждать что что-то наладится, но ничего не наладится же. Ну даже если наладится то не скоро, рада что могла себе обеспечить эту финансовую подушку. Я реально уже думала что можно и начать инвестировать но блин, реально никогда не знаешь, теперь думаю что можно было и пол года назад параллельно с подушкой все делать, но так иногда радует что хоть стабильность в деньгах есть, я все таки смогла научиться управлять немного своими финансами. Ну и новую работу искать, ну и что я тут найду? Наверно скорее всего нужно будет учить английский, вряд ли украинские компании сейчас смогут дать хорошую оплату, надо искать что-то онлайн, стабильное. Хотя какое оно стабильное, надо открывать свое дело в интернете, может делать магазин с доставкой? Да я так и не смогла этот английский выучить сколько уже ходила на курсы. Короче искать работу это мне вообще как настоящее испытание, ну и не думаю что без английского я кому-то нужна. Пока живем в бесплатном жилье, но тоже непонятно надолго это, вообще нужно ли возвращаться, в Киеве хотя бы есть дом, родные стены, ну и удобства все под рукой, но я боюсь ходить по улице вижу везде эти новости про растяжки, бомбы в молоке. Но и помощи от волонтеров больше есть и помощь с едой и вообще дома как то проще, здесь никого не знаю, но и знакомые если честно все уже разъехались и реально жизнь как с нуля. С мужем тоже сложно, постоянно приходится принимать решения, а он как будто оттягивает, но я же тоже не могу все время все делать. Короче мне реально не понятно как вообще быть, я боюсь что-то снова начинать, потому что не понятно на долго или скоро закончится."
4,"Психолог: Давайте выделим вопросы, которые я услышала и продолжим работу. Сейчас у вас нет основной работы для вас большой стресс искать новую, еще вы хотите вернуться в Киев, но не можете решить для себя стоит ли это сейчас делать, также вам хочется получить поддержку от мужа, но вы не можете получить той помощи на которую рассчитываете. Но самым большим испытанием для вас является то, что необходимо начать все с нуля. верно?"
5,"Ольга: Да, но даже не сам факт начала. Понимаете я уже несколько раз все теряла и начинала ""жизнь с этого нуля"". Поэтому мне кажется тут у меня что-то вроде иммунитета. Страх не с самой необходимости, я просто реально понимаю что если я еще раз все потеряю мне может просто не хотеться что-то делать снова. Именно когда живешь в период от нуля до ""опять все хочется"" тут самое страшное. Ощущение, что теряешь время, которого и так постоянно мало, и параллельно же ничего не улучшается, тебе еще наваливается сверху, и

In [207]:
conversation_df['interlocutor'] = conversation_df.apply(lambda x: 'cli' if x.text.lower().startswith('о') else 'psy', axis=1)
conversation_df['text'] = conversation_df['text'].apply(lambda x: x.strip().split(' ', 1)[1] if x.strip().lower().startswith('психолог') | x.strip().lower().startswith('о') else x.strip())
psy_df = conversation_df[conversation_df['interlocutor'] == 'psy']
cli_df = conversation_df[conversation_df['interlocutor'] == 'cli']
conversation_df.style.set_properties(**{'text-align': 'left'})

,text,interlocutor
0,"Добрый день, меня зовут Ирина. Ольга, у нас с вами на сегодня назначена консультация. Надеюсь, что наш разговор будет приятным и полезным. Расскажите с каким запросом вы пришли на нашу первую встречу?",psy
1,"Добрый день, это моя первая консультация в жизни. Я не особо понимаю что именно нужно говорить. Но у меня накопилось достаточно много всего и мне просто хотелось поговорить о том, что сейчас происходит с моей жизнью, ну много чего, конечно, как и у всех, все таки война, переезды, куча всего короче. Просто хотелось чтобы кто-то услышал, помог мне немного разобраться в себе.",cli
2,"Ольга, действительно сейчас многие проживают события к которым сложно быть готовым. Давайте начнем понемногу разбираться. Расскажите о себе. Где вы сейчас находитесь, в каких условиях живете, какие события происходят вокруг вас?",psy
3,"Сейчас я чувствую себя относительно в безопасности. Мы переехали из Киева на западную Украину. Здесь можно сказать, что тихо, но я не особо понимаю что происходит. Мне словно приходится начинать все с нуля. Деньги немного есть, ээээ ну там на пару месяцев хватит, но нужно искать работу. Не пойму нужно учится на новую специальность или попытаться найти работу менеджером, как раньше? Хотя кому эти менеджеры нужны, ничего толком не работает, наверно повезло людям которые сохранили работу, прям как-то радостно за них. Потерять финансовую стабильность мне реально тяжело, ждать что что-то наладится, но ничего не наладится же. Ну даже если наладится то не скоро, рада что могла себе обеспечить эту финансовую подушку. Я реально уже думала что можно и начать инвестировать но блин, реально никогда не знаешь, теперь думаю что можно было и пол года назад параллельно с подушкой все делать, но так иногда радует что хоть стабильность в деньгах есть, я все таки смогла научиться управлять немного своими финансами. Ну и новую работу искать, ну и что я тут найду? Наверно скорее всего нужно будет учить английский, вряд ли украинские компании сейчас смогут дать хорошую оплату, надо искать что-то онлайн, стабильное. Хотя какое оно стабильное, надо открывать свое дело в интернете, может делать магазин с доставкой? Да я так и не смогла этот английский выучить сколько уже ходила на курсы. Короче искать работу это мне вообще как настоящее испытание, ну и не думаю что без английского я кому-то нужна. Пока живем в бесплатном жилье, но тоже непонятно надолго это, вообще нужно ли возвращаться, в Киеве хотя бы есть дом, родные стены, ну и удобства все под рукой, но я боюсь ходить по улице вижу везде эти новости про растяжки, бомбы в молоке. Но и помощи от волонтеров больше есть и помощь с едой и вообще дома как то проще, здесь никого не знаю, но и знакомые если честно все уже разъехались и реально жизнь как с нуля. С мужем тоже сложно, постоянно приходится принимать решения, а он как будто оттягивает, но я же тоже не могу все время все делать. Короче мне реально не понятно как вообще быть, я боюсь что-то снова начинать, потому что не понятно на долго или скоро закончится.",cli
4,"Давайте выделим вопросы, которые я услышала и продолжим работу. Сейчас у вас нет основной работы для вас большой стресс искать новую, еще вы хотите вернуться в Киев, но не можете решить для себя стоит ли это сейчас делать, также вам хочется получить поддержку от мужа, но вы не можете получить той помощи на которую рассчитываете. Но самым большим испытанием для вас является то, что необходимо начать все с нуля. верно?",psy
5,"Да, но даже не сам факт начала. Понимаете я уже несколько раз все теряла и начинала ""жизнь с этого нуля"". Поэтому мне кажется тут у меня что-то вроде иммунитета. Страх не с самой необходимости, я просто реально понимаю что если я еще раз все потеряю мне может просто не хотеться что-то делать снова. Именно когда живешь в период от нуля до ""опять все хочется"" тут самое страшное. Ощущение, что теряешь время, которого и так постоянно мало, и параллельно же ничего не улучшается, тебе еще наваливается сверху, и нужно разгребать 

In [208]:
full_cli_text = ' '.join(i for i in cli_df.text.tolist()).strip()
full_psy_text = ' '.join(i for i in psy_df.text.tolist()).strip()
full_psy_text 

'Добрый день, меня зовут Ирина. Ольга, у нас с вами на сегодня назначена консультация. Надеюсь, что наш разговор будет приятным и полезным. Расскажите с каким запросом вы пришли на нашу первую встречу? Ольга, действительно сейчас многие проживают события к которым сложно быть готовым. Давайте начнем понемногу разбираться. Расскажите о себе. Где вы сейчас находитесь, в каких условиях живете, какие события происходят вокруг вас? Давайте выделим вопросы, которые я услышала и продолжим работу. Сейчас у вас нет основной работы для вас большой стресс искать новую, еще вы хотите вернуться в Киев, но не можете решить для себя стоит ли это сейчас делать, также вам хочется получить поддержку от мужа, но вы не можете получить той помощи на которую рассчитываете. Но самым большим испытанием для вас является то, что необходимо начать все с нуля. верно? решение есть всегда, задайте себе вопрос: готовы ли брать ответственность за принятые решения на себя? Расскажите подробнее, я так понимаю у вас раз

# Abstract Summarization

## Simple Baseline Solution (bert2bert)

bert2bert model, initialized with the DeepPavlov/rubert-base-cased pretrained weights and fine-tuned on the first 99% of "Rossiya Segodnya" news dataset for 2 epochs.


In [209]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

MODEL_NAME = "dmitry-vorobiev/rubert_ria_headlines"

bert2bert_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
bert2bert_model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']


In [210]:
cli_df.text.tolist()[2]

'Да, но даже не сам факт начала. Понимаете я уже несколько раз все теряла и начинала "жизнь с этого нуля". Поэтому мне кажется тут у меня что-то вроде иммунитета. Страх не с самой необходимости, я просто реально понимаю что если я еще раз все потеряю мне может просто не хотеться что-то делать снова. Именно когда живешь в период от нуля до "опять все хочется" тут самое страшное. Ощущение, что теряешь время, которого и так постоянно мало, и параллельно же ничего не улучшается, тебе еще наваливается сверху, и нужно разгребать все одновременно. Теряешь дом, нет людей рядом, начинаются долги, нет уверенности в своих силах, моложе не становишься, ну и вечная конкуренция потому что у людей еще есть запал, а тебе 35 и ты такой здрасьте, я хочу заявить о себе. Я начинаю постоянно думать в какой именно момент я перестала контролировать свою жизнь? Ведь мне нравилось жить в том потоке, что происходил. Мне реально часто везло, но прям про будущее я не думала. А теперь читаю про блекауты, теории за

In [211]:
def extract_summary_bert2bert(text):
  encoded_batch = bert2bert_tokenizer.prepare_seq2seq_batch(
    [text],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512)

  output_ids = bert2bert_model.generate(
    input_ids=encoded_batch["input_ids"],
    max_length=36,
    no_repeat_ngram_size=3,
    num_beams=5,
    top_k=0
  )

  headline = bert2bert_tokenizer.decode(output_ids[0], 
                            skip_special_tokens=True, 
                            clean_up_tokenization_spaces=False)
  return headline

In [212]:
def get_summary_phrases(context_list, func):
  sums = []
  for i in context_list:
    sums.append(func(i))
  return sums

In [213]:
bert2bert_cli_sums = get_summary_phrases(cli_df.text.tolist(), extract_summary_bert2bert)
bert2bert_psy_sums = get_summary_phrases(psy_df.text.tolist(), extract_summary_bert2bert)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3516: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


## Simple Baseline Solution (T5 pretrained on Telegram dataset)
RuT5TelegramHeadlines

In [214]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

model_name = "IlyaGusev/rut5_base_headline_gen_telegram"
t5_tokenizer = AutoTokenizer.from_pretrained(model_name)
t5_model = T5ForConditionalGeneration.from_pretrained(model_name)

In [215]:
def extract_summary_t5(article_text):
  input_ids = t5_tokenizer(
    [article_text],
    max_length=600,
    add_special_tokens=True,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
  )["input_ids"]

  output_ids = t5_model.generate(
    input_ids=input_ids
  )[0]

  headline = t5_tokenizer.decode(output_ids, skip_special_tokens=True)
  return headline

In [216]:
t5_cli_sums = get_summary_phrases(cli_df.text.tolist(), extract_summary_t5)
t5_psy_sums = get_summary_phrases(psy_df.text.tolist(), extract_summary_t5)

## SimpleBaseline Solution (T5 pretrained on Gazeta dataset)

In [217]:
model_name_t5_gazeta = "IlyaGusev/rut5_base_sum_gazeta"
tokenizer_t5_gazeta = AutoTokenizer.from_pretrained(model_name_t5_gazeta)
model_t5_gazeta = T5ForConditionalGeneration.from_pretrained(model_name_t5_gazeta)


In [218]:
def extract_summary_t5_gazeta(article_text):
  input_ids = tokenizer_t5_gazeta(
    [article_text],
    max_length=600,
    add_special_tokens=True,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
  )["input_ids"]

  output_ids = model_t5_gazeta.generate(
    input_ids=input_ids,
    no_repeat_ngram_size=4
  )[0]

  summary = tokenizer_t5_gazeta.decode(output_ids, skip_special_tokens=True)
  return summary

In [219]:
t5_cli_sums_gazeta = get_summary_phrases(cli_df.text.tolist(), extract_summary_t5_gazeta)
t5_psy_sums_gazeta = get_summary_phrases(psy_df.text.tolist(), extract_summary_t5_gazeta)

# Evaluation 

Results with bert2bert 

In [220]:
print(bert2bert_cli_sums)
print(bert2bert_psy_sums)

['татьяна сергеева', 'украина в безопасности, или как я люблю работу с англиискими деньгами', 'от нуля к блекаутым', 'сергеи', '" я не хочу ехать в киев, но через неделю "', '" я не хочу жить однои, а мы уже не будем жить "', '" я хочу быть вместе с мужем "', '" я уже не знаю, что я люблю, а я люблю себя? "', 'сергеи']
['сергеи', 'ольга, ольга', '" я не знаю, что нужно начать все с нуля "', 'ответственность за принятые решения на себя?', 'что делать с мужем?', 'о том, что муж оттягивал принятие решении решении решении', '" я не нужно никого ждать... "', 'как правильно договориться с семьеи. треилер фильма', '" я хочу разобраться, что вы чувствуетеся в эмоциях "', 'сергеи сергеевич']


Results with T5_telegram

In [221]:
print(t5_cli_sums)
print(t5_psy_sums)

['Добрый день, моя первая консультация в жизни', 'Что делать, если наладится ничего не наладится, но ничего не наладится', 'Страх не с самой необходимости, я уже несколько раз все теряла и начинала "жизнь с этого нуля"', 'Не надо ждать коллективного решения, а сам тихо сделала и ни у кого не спрашиваешь', 'Что делать, если бы муж помог мне и согласился бы на переезд уже было бы проще', 'В постоянном ожидании чуда жить тоже не легко', 'Не хочу представлять, а если бы. Мне нравятся наши отношения, хотя такие штуки когда приходится ждать решения прям бесят', 'Я как будто живу с бомбой с часовым механизмом и даже не предлагаю никаких идей', 'Спасибо, реально интересно чего я чувствую иногда кажется что уже ничего']
['У нас с вами назначена консультация', 'Что происходит вокруг вас?', 'Что нужно знать о работе с нуля?', 'Вопрос: готовы ли брать ответственность за принятые решения на себя?', 'Вопрос о разводе с мужем: я так понимаю у вас разные точки зрения на какую то ситуацию с мужем?', 'М

Results with T5_gazeta

In [224]:
print(t5_cli_sums_gazeta)
print(t5_psy_sums_gazeta)

['Мне хотелось поговорить о том, что сейчас происходит с моей жизнью, а также о том как происходит сейчас. Я не особо понимаю, что именно нужно говорить. Просто хотелось чтобы кто-то услышал, помог мне разобраться в себе.', 'Я живу в Киеве и хочу найти работу в интернете, но не понимаю, почему я живу в этом городе, а не в стране, а в стране очень тихо и спокойно.', 'Я уже несколько раз все теряла и начинала жить в период от нуля до «опять все хочется». Мне кажется, что если я еще раз все потеряю, мне может просто не хотеться что-то делать снова.', 'Я пытаюсь балансировать между личной ответственностью за собственную жизнь и умением договориться. Не надо ждать коллективного решения, а сам тихо сделала и ни у кого не спрашивает.', 'Я не хочу ехать в Киев, я хочу чтобы мы вдвоем поехали, потому что завтра лучше не станет, а после завтра тоже тоже станет.', 'Я не хочу жить одна, а хочу понять, как можно было бы жить, если бы мы не были вместе, потому что я не хотела бы постоянно ждать, где